# Исследование надежности заемщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

**Содержание**<a id='toc0_'></a>    
- 1. [Открытие таблицы и изучение общей информации о данных](#toc1_)    
- 2. [Предобработка данных](#toc2_)    
  - 2.1. [Удаление пропусков](#toc2_1_)    
  - 2.2. [Обработка аномальных значений](#toc2_2_)    
  - 2.3. [Удаление пропусков (продолжение)](#toc2_3_)    
  - 2.4. [Изменение типов данных](#toc2_4_)    
  - 2.5. [Обработка дубликатов](#toc2_5_)    
  - 2.6. [Категоризация данных](#toc2_6_)    
- 3. [Исследуем данные и ответим на вопросы](#toc3_)    
  - 3.1. [Связь между количеством детей и возвратом кредита в срок](#toc3_1_)    
  - 3.2. [Связь между семейным положением и возвратом кредита в срок](#toc3_2_)    
  - 3.3. [Связь между уровнем дохода и возвратом кредита в срок](#toc3_3_)    
  - 3.4. [Влияние цели кредита на его возврат в срок](#toc3_4_)    
  - 3.5. [Возможные причины появления пропусков в исходных данных.](#toc3_5_)    
- 4. [Общий вывод.](#toc4_)    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=false
	minLevel=2
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

## 1. <a id='toc1_'></a>[Открытие таблицы и изучение общей информации о данных](#toc0_)

**Импорт библиотеки pandas. Считывание данных из csv-файла в датафрейм и сохранение в переменную `data`.**

In [1]:
import pandas as pd

try:
    data = pd.read_csv('data.csv')
except FileNotFoundError:
    data = pd.read_csv('url')

**Первые 20 строчек датафрейма `data`.**

In [2]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**Основная информация о датафрейме.**

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## 2. <a id='toc2_'></a>[Предобработка данных](#toc0_)

### 2.1. <a id='toc2_1_'></a>[Удаление пропусков](#toc0_)

**Количество пропущенных значений для каждого столбца.**

In [4]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

**В двух столбцах есть пропущенные значения. Один из них — `days_employed`. Пропуски в этом столбце будут обработаны на следующем этапе. Другой столбец с пропущенными значениями — `total_income` — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу из столбца `income_type`. Например, у человека с типом занятости `сотрудник` пропуск в столбце `total_income` должен быть заполнен медианным доходом среди всех записей с тем же типом.**

In [5]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### 2.2. <a id='toc2_2_'></a>[Обработка аномальных значений](#toc0_)

**В данных могут встречаться артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке. Таким артефактом будет отрицательное количество дней трудового стажа в столбце `days_employed`. Для реальных данных это нормально. Необходимо обработать значения в этом столбце: заменить все отрицательные значения положительными с помощью метода `abs()`.**

In [6]:
data['days_employed'] = data['days_employed'].abs()

**Медианное значение трудового стажа для каждого типа занятости `days_employed` в днях.**

In [7]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставим их как есть. Тем более этот столбец не понадобится для исследования.

**Выведем перечень уникальных значений столбца `children`.**

In [8]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

**В столбце `children` есть два аномальных значения. Удалим строки, в которых встречаются такие аномальные значения из датафрейма `data`.**

In [9]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

**Ещё раз выведем перечень уникальных значений столбца `children`, чтобы убедиться, что артефакты удалены.**

In [10]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5], dtype=int64)

### 2.3. <a id='toc2_3_'></a>[Удаление пропусков (продолжение)](#toc0_)

**Заполним пропуски в столбце `days_employed` медианными значениями по каждому типу занятости `income_type`.**

In [11]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

**Убедимся, что все пропуски заполнены. Проверим себя и ещё раз выведите количество пропущенных значений для каждого столбца.**

In [12]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### 2.4. <a id='toc2_4_'></a>[Изменение типов данных](#toc0_)

**Заменим вещественный тип данных в столбце `total_income` на целочисленный с помощью метода `astype()`.**

In [13]:
data['total_income'] = data['total_income'].astype(int)

### 2.5. <a id='toc2_5_'></a>[Обработка дубликатов](#toc0_)

**Обработаем неявные дубликаты в столбце `education`. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведём их к нижнему регистру.**

In [14]:
data['education'] = data['education'].str.lower()

**Выведем на экран количество строк-дубликатов в данных. Если такие строки присутствуют, удалим их.**

In [15]:
data.duplicated().sum()

71

In [16]:
data = data.drop_duplicates()

### 2.6. <a id='toc2_6_'></a>[Категоризация данных](#toc0_)

**На основании диапазонов, указанных ниже, создадим в датафрейме `data` столбец `total_income_category` с категориями:**

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.


**Например, кредитополучателю с доходом 25000 нужно назначить категорию `'E'`, а клиенту, получающему 235000, — `'B'`. Используем собственную функцию с именем `categorize_income()` и метод `apply()`.**

In [17]:
def categorize_income(income):
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'

In [18]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

**Выведем на экран перечень уникальных целей взятия кредита из столбца `purpose`.**

In [19]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

**Создадим функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:**

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

**Например, если в столбце `purpose` находится подстрока `'на покупку автомобиля'`, то в столбце `purpose_category` должна появиться строка `'операции с автомобилем'`.**

**Используем собственную функцию с именем `categorize_purpose()` и метод `apply()`. Изучим данные в столбце `purpose` и определим, какие подстроки помогут правильно определить категорию.**

In [20]:
def categorize_purpose(row):
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
        return 'нет категории'

In [21]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

## 3. <a id='toc3_'></a>[Исследуем данные и ответим на вопросы](#toc0_)

### 3.1. <a id='toc3_1_'></a>[Связь между количеством детей и возвратом кредита в срок](#toc0_)

**Напишем функцию, создающую сводную таблицу по заданному столбцу.**

In [22]:
columns_new_names = ['Доля должников', 'Всего кредитополучателей', 'Всего должников']
def create_pivot_table(column):
    table = pd.pivot_table(data, values='debt', index=column, aggfunc=['mean', 'count', 'sum'])
    table.columns = pd.Index(columns_new_names)
    return table

**Сгруппируем заёмщиков в зависимости от количества детей:
`count` - общее количество заёмщиков
`sum` - общее число должников
`mean` - относительное число должников**

In [23]:
# Группировка и подсчёт
data_children_grouped = data.groupby('children')['debt'].agg(['mean', 'count', 'sum'])
display(data_children_grouped)

,mean,count,sum
children,,,
0,0.075438,14091,1063
1,0.092346,4808,444
2,0.094542,2052,194
3,0.081818,330,27
4,0.097561,41,4
5,0.000000,9,0


**Т.к. данных по многодетным (3 и более детей) относительно немного, выделим их в отдельную группу и вычислим среднее число должников.**

In [24]:
data_children_grouped[3:].sum()['sum'] / data_children_grouped[3:].sum()['count']

0.08157894736842106

**Представим результат в виде сводной таблицы.**

In [25]:
create_pivot_table(column='children')

,Доля должников,Всего кредитополучателей,Всего должников
children,,,
0,0.075438,14091,1063
1,0.092346,4808,444
2,0.094542,2052,194
3,0.081818,330,27
4,0.097561,41,4
5,0.000000,9,0


***Вывод:* *бездетные* возвращают кредит в срок чаще остальных, *многодетные* (3-5 детей) немного реже, а больше всего должников среди *малодетных (1-2 ребёнка).***

### 3.2. <a id='toc3_2_'></a>[Связь между семейным положением и возвратом кредита в срок](#toc0_)

**Сгруппируем заёмщиков в зависимости от семейного положения:
`count` - общее количество заёмщиков
`sum` - общее число должников
`mean` - относительное число должников**

In [26]:
# Группировка и подсчёт
data.groupby('family_status')['debt'].agg(['mean','count', 'sum']).sort_values(by='mean')

,mean,count,sum
family_status,,,
вдовец / вдова,0.066246,951,63
в разводе,0.070648,1189,84
женат / замужем,0.075606,12261,927
гражданский брак,0.093130,4134,385
Не женат / не замужем,0.097639,2796,273


**Представим результат в виде сводной таблицы.**

In [27]:
create_pivot_table(column='family_status')

,Доля должников,Всего кредитополучателей,Всего должников
family_status,,,
Не женат / не замужем,0.097639,2796,273
в разводе,0.070648,1189,84
вдовец / вдова,0.066246,951,63
гражданский брак,0.093130,4134,385
женат / замужем,0.075606,12261,927


***Вывод:* те, кто *состоят (или состояли) в браке*, становятся должниками заметно реже других.**

### 3.3. <a id='toc3_3_'></a>[Связь между уровнем дохода и возвратом кредита в срок](#toc0_)

**Сгруппируем заёмщиков в зависимости от уровня дохода.**

In [28]:
# Группировка и подсчёт
data.groupby('total_income_category')['debt'].agg(['mean', 'count', 'sum'])

,mean,count,sum
total_income_category,,,
A,0.080000,25,2
B,0.070602,5014,354
C,0.084982,15921,1353
D,0.060172,349,21
E,0.090909,22,2


**Представим результат в виде сводной таблицы.**

In [29]:
create_pivot_table(column='total_income_category')

,Доля должников,Всего кредитополучателей,Всего должников
total_income_category,,,
A,0.080000,25,2
B,0.070602,5014,354
C,0.084982,15921,1353
D,0.060172,349,21
E,0.090909,22,2


**Данных по категориям A, D, E недостаточно, а основываться только на данных о двух категориях - C и D - не совсем корректно. Попробуем разделить категорию C на подкатегории С1, С2, С3 и проверить гипотезу.**

In [30]:
category_c = data.loc[data['total_income_category'] == 'C']
c_subcategories = pd.cut(category_c['total_income'], bins=3, labels=['C3', 'C2', 'C1'])
sub_data = pd.DataFrame({'subcategory' : c_subcategories, 'debt' : category_c['debt'] })
sub_result = sub_data.groupby('subcategory')['debt'].agg(['mean', 'count', 'sum'])
sub_result.columns = pd.Index(['Доля должников', 'Всего кредитополучателей', 'Всего должников'])
sub_result

,Доля должников,Всего кредитополучателей,Всего должников
subcategory,,,
C3,0.081247,4074,331
C2,0.087073,7109,619
C1,0.085057,4738,403


**Число должников в полученных подкатегориях примерно равно.**

***Вывод:* заёмщики с более высоким уровнем дохода реже становятся должниками**

### 3.4. <a id='toc3_4_'></a>[Влияние цели кредита на его возврат в срок](#toc0_)

**Сгруппируем заёмщиков в зависимости от уровня дохода аналогично предыдущим пунктам.**

In [31]:
#Группировка и вычисления
data.groupby('purpose_category')['debt'].agg(['mean', 'count', 'sum']).sort_values(by='mean')

,mean,count,sum
purpose_category,,,
операции с недвижимостью,0.072551,10751,780
проведение свадьбы,0.079118,2313,183
получение образования,0.092528,3988,369
операции с автомобилем,0.093480,4279,400


**Представим результат в виде сводной таблицы.**

In [32]:
create_pivot_table(column='purpose_category')

,Доля должников,Всего кредитополучателей,Всего должников
purpose_category,,,
операции с автомобилем,0.093480,4279,400
операции с недвижимостью,0.072551,10751,780
получение образования,0.092528,3988,369
проведение свадьбы,0.079118,2313,183


***Вывод:* число должников в категориях *операции с недвижимостью* и *проведение свадьбы* меньше, чем в категориях *получение образования* и *операции с автомобилем.***

### 3.5. <a id='toc3_5_'></a>[Возможные причины появления пропусков в исходных данных.](#toc0_)

***Ответ:* Пропуски в данных могут быть следствием двух факторов:
*Человеческий* - люди не предоставили некоторые данные или этого не требовалось
*Технический* - возникли ошибки при загрузке и/или выгрузке данных**

## 4. <a id='toc4_'></a>[Общий вывод.](#toc0_)

**Изучен полученный датасет, удалены пропуски и дубликаты, при необходимости типы данных изменены, сами данные категоризованы, высказаны предположения о причинах появления пропусков.
По различным типам группировки выявлены наиболее и наименее платёжеспособные категории заёмщиков:**
<br>

| Тип группировки        | Лучшие плательщики         | Самые частые должники            |
|:-----------------------|:---------------------------|:---------------------------------|
| По количеству детей    | Бездетные                  | Малодетные (1-2 ребёнка)         |
| По семейному положению | Состоят (состояли) в браке | Не состоят (не состояли в браке) |
| По уровню дохода       | Высокий доход              | Низкий доход                     |
| По цели кредита        | Операции с недвижимостью   | Операции с автомобилем           |